In [ ]:
from pinkylib import Camera, Pinky
import cv2
import numpy as np
import time

# Camera와 Pinky 객체 생성
cam = Camera()
pinky = Pinky()

# 카메라 시작
cam.start()
pinky.enable_motor()
pinky.start_motor()

# 화면 크기 설정
WIDTH = 640
HEIGHT = 480

# Bird's Eye View 설정 값
h1 = 0.725
h2 = 0.4
w1 = 0.25

# Bird's Eye View 변환 함수
def bird_eye_view(img):
    src_point = np.array(
        [[0, h1 * HEIGHT], [WIDTH, h1 * HEIGHT], [w1 * WIDTH, h2 * HEIGHT], [(1 - w1) * WIDTH, h2 * HEIGHT]],
        dtype=np.float32
    )
    dst_point = np.array([[0, HEIGHT], [WIDTH, HEIGHT], [0, 0], [WIDTH, 0]], dtype=np.float32)
    matrix = cv2.getPerspectiveTransform(src_point, dst_point)
    dst = cv2.warpPerspective(img, matrix, (WIDTH, HEIGHT))
    return dst

# Canny Edge Detection 함수 (Gaussian Blur와 Dilation 포함)
def canny_detector(img, low_threshold=50, high_threshold=150):
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    edges = cv2.Canny(blurred, low_threshold, high_threshold)
    dilated_edges = cv2.dilate(edges, np.ones((3, 3), np.uint8), iterations=1)
    return dilated_edges

# 수평선 제거 함수 (허프 변환 사용)
def remove_horizontal_edges(edges):
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=20)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
            # 각도가 0~10도 또는 170~180도 사이이면 수평으로 간주하고 제거
            if angle < 10 or angle > 170:
                cv2.line(edges, (x1, y1), (x2, y2), (0, 0, 0), 5)
    return edges

# 이전 차선 중심 위치를 저장할 변수와 허용되는 미검출 프레임 수
last_lane_center_x = None
missed_frames = 0
max_missed_frames = 5  # 차선이 감지되지 않아도 유지할 프레임 수

try:
    while True:
        # 카메라로부터 프레임을 가져옴
        frame = cam.get_frame()
        
        # Bird's Eye View 적용
        bird_view = bird_eye_view(frame)
        
        # Canny Edge Detection 수행
        edges = canny_detector(bird_view)
        
        # 수평선 제거
        edges_no_horizontal = remove_horizontal_edges(edges)
        
        # 차선의 중심을 찾기 위해 이미지의 아래쪽 부분에 스캔라인을 설정
        scan_line_y = edges_no_horizontal.shape[0] - 50  # 아래쪽 50픽셀 위에 스캔라인 설정
        scan_line = edges_no_horizontal[scan_line_y, :]

        # 차선의 왼쪽과 오른쪽 가장자리 찾기
        left_line_x = None
        right_line_x = None

        for x in range(WIDTH // 2):
            if scan_line[x] > 0:
                left_line_x = x
                break

        for x in range(WIDTH - 1, WIDTH // 2, -1):
            if scan_line[x] > 0:
                right_line_x = x
                break

        # 차선이 감지되지 않았을 경우
        if left_line_x is None and right_line_x is None:
            
            pinky.stop()
            last_lane_center_x = None  # 마지막 차선 위치 초기화
            continue
            
        else:
            # 차선이 감지되면 미검출 프레임 수 초기화하고 차선 중심 계산
            missed_frames = 0
            if left_line_x is not None and right_line_x is not None:
                lane_center_x = (left_line_x + right_line_x) // 2
                last_lane_center_x = lane_center_x  # 현재 차선 중심 위치를 저장

        frame_center_x = WIDTH // 2

        # 차선 중심과 화면 중심의 차이 계산
        dx = lane_center_x - frame_center_x
        
        # dx에 따라 모터 속도 조정
        base_speed = 60  # 기본 속도를 낮춤
        adjustment = 7   # 조정 속도도 줄임

        if abs(dx) < 10:
            # 거의 중앙에 있으면 직진
            left_speed = base_speed
            right_speed = base_speed
        elif dx > 0:
            # 오른쪽으로 치우쳐 있으면 왼쪽 바퀴 속도 증가
            left_speed = base_speed + adjustment
            right_speed = base_speed - adjustment
        else:
            # 왼쪽으로 치우쳐 있으면 오른쪽 바퀴 속도 증가
            left_speed = base_speed - adjustment
            right_speed = base_speed + adjustment

        # 모터 움직임
        pinky.move(left_speed, right_speed)

        # 두 개의 화면을 하나로 합침 (좌: 원본, 우: 수평선 제거 후 엣지)
        combined_display = np.hstack((frame, cv2.cvtColor(edges_no_horizontal, cv2.COLOR_GRAY2BGR)))

        # 결과 화면 표시
        cam.display_jupyter(combined_display)

        # 'q' 키로 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # 종료 시 모터와 카메라 정리
    pinky.stop()
    pinky.disable_motor()
    pinky.stop_motor()
    pinky.clean()
    cam.close()
